In [1]:
import glob
from os.path import dirname, basename
import pandas as pd
import numpy as np
from inpainting.load import load_dataframe
from inpainting.utils import get_paths


In [2]:
data_dir = '../data'
results_dirs = get_paths('../results/image_inpainting/*')

times_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_df = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(times_df, results_df, on=['model', 'sequence_name', 'frame_id'])
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df

,frame_id,inference_time,model,sequence_name,mean_squared_error,peak_signal_noise_ratio,structural_similarity,object_sequence_name,object_id
0,0,61.071457,ii_deepfillv1,bear_paragliding-launch_002,8.538185,38.817148,0.986603,paragliding-launch,2
1,1,61.265408,ii_deepfillv1,bear_paragliding-launch_002,6.608233,39.929950,0.987867,paragliding-launch,2
2,2,61.270622,ii_deepfillv1,bear_paragliding-launch_002,9.268295,38.460805,0.986352,paragliding-launch,2
3,3,55.175041,ii_deepfillv1,bear_paragliding-launch_002,12.210953,37.263308,0.984801,paragliding-launch,2
4,4,37.858974,ii_deepfillv1,bear_paragliding-launch_002,18.366338,35.490578,0.983378,paragliding-launch,2
...,...,...,...,...,...,...,...,...,...
41036,28,122.652321,ii_regionfill,walking_judo_001,101.306847,28.074416,0.969807,judo,1
41037,29,113.137665,ii_regionfill,walking_judo_001,115.875987,27.490869,0.971334,judo,1
41038,30,120.471298,ii_regionfill,walking_judo_001,45.878459,31.514715,0.971554,judo,1
41039,31,114.824928,ii_regionfill,walking_judo_001,55.003652,30.726888,0.963900,judo,1


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
# metrics['inference_time'] = df['inference_time']
metrics['mean_squared_error'] = df['mean_squared_error']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio']
metrics['structural_similarity'] = df['structural_similarity']
metrics.groupby('model').mean()

,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,
ii_deepfillv1,308.047086,31.525548,0.939509
ii_deepfillv1_flownet2,248.407427,36.360728,0.957652
ii_deepfillv1_pwcnet,248.049171,36.388265,0.957853
ii_deepfillv2,286.156313,30.975536,0.940481
ii_kernelfill,280.903865,33.261293,0.948482
ii_pconvunet,461.925188,27.706346,0.928443
ii_regionfill,250.708970,34.563039,0.950062
